In [1]:
pip install requests tabulate 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import json
import pandas as pd
from tabulate import tabulate

# Base API URL
BASE_URL = "https://api.fda.gov/drug/"

def fetch_data(endpoint, params):
    """Generic function to fetch data from OpenFDA API"""
    try:
        response = requests.get(f"{BASE_URL}{endpoint}.json", params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def format_label_data(data):
    """Format drug labeling data into structured sections"""
    if not data.get("results"):
        return "No results found."
    
    drug = data["results"][0]
    openfda = drug.get("openfda", {})
    
    formatted = {
        "Basic Info": {
            "Brand Name": ", ".join(openfda.get("brand_name", ["N/A"])),
            "Generic Name": ", ".join(openfda.get("generic_name", ["N/A"])),
            "Manufacturer": ", ".join(openfda.get("manufacturer_name", ["N/A"])),
            "NDC Code": ", ".join(openfda.get("product_ndc", ["N/A"])),
            "Dosage Form": ", ".join(drug.get("dosage_form", ["N/A"]))
        },
        "Medical Use": {
            "Purpose": drug.get("indications_and_usage", ["N/A"])[0] if isinstance(drug.get("indications_and_usage"), list) else drug.get("indications_and_usage", "N/A"),
            "Target Conditions": drug.get("indications", ["N/A"])[0] if isinstance(drug.get("indications"), list) else drug.get("indications", "N/A"),
            "Mechanism of Action": drug.get("description", "N/A") or drug.get("clinical_pharmacology", "N/A")
        },
        "Safety & Warnings": {
            "Side Effects": drug.get("adverse_reactions", ["N/A"])[0] if isinstance(drug.get("adverse_reactions"), list) else drug.get("adverse_reactions", "N/A"),
            "Contraindications": drug.get("contraindications", ["N/A"])[0] if isinstance(drug.get("contraindications"), list) else drug.get("contraindications", "N/A"),
            "Black Box Warnings": drug.get("boxed_warning", ["N/A"])[0] if isinstance(drug.get("boxed_warning"), list) else drug.get("boxed_warning", "N/A"),
            "Pregnancy Risk": drug.get("pregnancy_category", "N/A")
        },
        "Usage Instructions": {
            "Dosage": drug.get("dosage_and_administration", ["N/A"])[0] if isinstance(drug.get("dosage_and_administration"), list) else drug.get("dosage_and_administration", "N/A"),
            "Storage": drug.get("storage_and_handling", ["N/A"])[0] if isinstance(drug.get("storage_and_handling"), list) else drug.get("storage_and_handling", "N/A"),
            "Overdose Handling": drug.get("overdosage", ["N/A"])[0] if isinstance(drug.get("overdosage"), list) else drug.get("overdosage", "N/A")
        }
    }
    return formatted

def format_adverse_events(data):
    """Format adverse event data"""
    if not data.get("results"):
        return "No adverse events found."
    
    formatted_events = []
    for event in data["results"]:
        patient = event.get("patient", {})
        formatted_events.append({
            "Reactions": ", ".join(patient.get("reaction", [{}])[0].get("reactionmeddrapt", ["N/A"])),
            "Severity": patient.get("reaction", [{}])[0].get("reactionoutcome", "N/A"),
            "Age": patient.get("patientonsetage", "N/A"),
            "Gender": patient.get("patientsex", "N/A"),
            "Drugs Involved": ", ".join([d.get("openfda", {}).get("brand_name", ["N/A"])[0] for d in patient.get("drug", []) if d.get("openfda", {}).get("brand_name")])
        })
    return formatted_events

def format_approval_data(data):
    """Format drug approval data"""
    if not data.get("results"):
        return "No approval data found."
    
    formatted = []
    for result in data["results"]:
        openfda = result.get("openfda", {})
        formatted.append({
            "Brand Name": ", ".join(openfda.get("brand_name", ["N/A"])),
            "Generic Name": ", ".join(openfda.get("generic_name", ["N/A"])),
            "Approval Date": result.get("approval_date", "N/A"),
            "Application Type": result.get("application_type", "N/A"),
            "Therapeutic Class": ", ".join(openfda.get("product_type", ["N/A"]))
        })
    return formatted

def print_formatted(data, title=""):
    """Print data in a formatted way"""
    if title:
        print(f"\n{'='*50}")
        print(f"{title.upper():^50}")
        print(f"{'='*50}")
    
    if isinstance(data, dict):
        for section, items in data.items():
            print(f"\n{section}:")
            for key, value in items.items():
                print(f"  • {key}: {value}")
    elif isinstance(data, list):
        print(tabulate(data, headers="keys", tablefmt="grid"))
    else:
        print(data)

def drug_labeling_menu():
    """Menu for drug labeling data"""
    print("\nDrug Labeling Search Options:")
    print("1. Search by Brand Name")
    print("2. Search by Generic Name")
    print("3. Search by NDC Code")
    print("4. Back to Main Menu")
    
    choice = input("Enter your choice (1-4): ")
    if choice == "4":
        return
    
    search_term = input("Enter search term: ")
    params = {"limit": 1}
    
    if choice == "1":
        params["search"] = f"openfda.brand_name:{search_term}"
    elif choice == "2":
        params["search"] = f"openfda.generic_name:{search_term}"
    elif choice == "3":
        params["search"] = f"openfda.product_ndc:{search_term}"
    else:
        print("Invalid choice")
        return
    
    data = fetch_data("label", params)
    if data:
        formatted = format_label_data(data)
        print_formatted(formatted, "Drug Labeling Information")

def adverse_events_menu():
    """Menu for adverse events data"""
    print("\nAdverse Events Search Options:")
    print("1. Search by Brand Name")
    print("2. Search by Reaction")
    print("3. Back to Main Menu")
    
    choice = input("Enter your choice (1-3): ")
    if choice == "3":
        return
    
    search_term = input("Enter search term: ")
    params = {"limit": 5}
    
    if choice == "1":
        params["search"] = f"patient.drug.openfda.brand_name:{search_term}"
    elif choice == "2":
        params["search"] = f"patient.reaction.reactionmeddrapt:{search_term}"
    else:
        print("Invalid choice")
        return
    
    data = fetch_data("event", params)
    if data:
        formatted = format_adverse_events(data)
        print_formatted(formatted, "Adverse Events Report")

def drug_approvals_menu():
    """Menu for drug approval data"""
    print("\nDrug Approvals Search Options:")
    print("1. Search by Brand Name")
    print("2. Search by Application Type")
    print("3. Back to Main Menu")
    
    choice = input("Enter your choice (1-3): ")
    if choice == "3":
        return
    
    search_term = input("Enter search term: ")
    params = {"limit": 5}
    
    if choice == "1":
        params["search"] = f"openfda.brand_name:{search_term}"
    elif choice == "2":
        params["search"] = f"application_type:{search_term}"
    else:
        print("Invalid choice")
        return
    
    data = fetch_data("drugsfda", params)
    if data:
        formatted = format_approval_data(data)
        print_formatted(formatted, "Drug Approval Information")

def main_menu():
    """Main menu"""
    while True:
        print("\n" + "="*50)
        print("OPENFDA DRUG DATA RETRIEVAL SYSTEM".center(50))
        print("="*50)
        print("1. Drug Labeling Information")
        print("2. Adverse Events Reports")
        print("3. Drug Approval Data")
        print("4. Exit")
        
        choice = input("\nEnter your choice (1-4): ")
        
        if choice == "1":
            drug_labeling_menu()
        elif choice == "2":
            adverse_events_menu()
        elif choice == "3":
            drug_approvals_menu()
        elif choice == "4":
            print("Exiting program...")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main_menu()


        OPENFDA DRUG DATA RETRIEVAL SYSTEM        
1. Drug Labeling Information
2. Adverse Events Reports
3. Drug Approval Data
4. Exit



Enter your choice (1-4):  1



Drug Labeling Search Options:
1. Search by Brand Name
2. Search by Generic Name
3. Search by NDC Code
4. Back to Main Menu


Enter your choice (1-4):  2
Enter search term:  ibuprofen



            DRUG LABELING INFORMATION             

Basic Info:
  • Brand Name: Ibuprofen Dye Free
  • Generic Name: IBUPROFEN
  • Manufacturer: CVS Pharmacy
  • NDC Code: 59779-438
  • Dosage Form: N/A

Medical Use:
  • Purpose: Uses temporarily relieves minor aches and pains due to: headache toothache backache menstrual cramps the common cold muscular aches minor pain of arthritis temporarily reduces fever
  • Target Conditions: N/A
  • Mechanism of Action: N/A

Safety & Warnings:
  • Side Effects: N/A
  • Contraindications: N/A
  • Black Box Warnings: N/A
  • Pregnancy Risk: N/A

Usage Instructions:
  • Dosage: Directions do not take more than directed the smallest effective dose should be used adults and children 12 years and over: take 1 tablet every 4 to 6 hours while symptoms persist if pain or fever does not respond to 1 tablet, 2 tablets may be used do not exceed 6 tablets in 24 hours, unless directed by a doctor children under 12 years: ask a doctor
  • Storage: Other inform


Enter your choice (1-4):  4


Exiting program...
